In [1]:
from ManageSets import *
import json, collections, copy
from datetime import datetime

# what is happening with the recalibrateTermsQueue module that makes it not recal all the terms?

In [2]:
# INITIALIZE A NEW SET 

# bring in a raw set
with open('test_set.json', 'r') as infile:
    raw_set = json.load(infile)

# initialize study set from raw_set
term_queues = createTermQueues(raw_set)
terms_dict = raw_set

s3_seen = len([term for term, term_data in terms_dict.items()
                    if 'seen' in term_data  # only s3 terms
                    and term_data['seen'] == True])

set_data = {'name' : 'test',
            'init_date' : datetime.now().__str__(),
            'deck_min' : 100,
            'cycle_length' : 60,
            'total_sessions' : 1,
            'term_queues' : term_queues,
            'terms_dict' : terms_dict}

# SIMULATE A STUDY SESSION WITH CHANGES

# generate a study deck
study_session = buildDeck(term_queues, s3_seen, set_data['deck_min'], set_data['cycle_length'])

deck = study_session['deck']

print(study_session['deck_stats'])
print('total: ', sum(stat for score, stat in study_session['deck_stats'].items()))

# generate a finite set of changes
changes = []
for i in range(1,11):
    new_scores = [0,1,2,3]
    for ns in new_scores:
        changes.append(str(ns))

print(f'initiating {len(changes)} changes')
        
# 'study' deck and generate random cluster of changes
for term in deck:
    if changes and terms_dict[term]['score'] != changes[0]:
        # change term score
        terms_dict[term]['score'] = changes.pop(0)

Counter({'1': 46, '2': 23, '0': 19, '3': 12})
total:  100
initiating 40 changes


In [3]:
def recalibrateTermQueues(terms_dict, t_queues):
    '''
    Adjusts term queues to the terms_dict when a term is changed to a new score.
    Terms are removed from their old queue and added to the new ones.
    New S3 go to the back of the S3 queue and are tagged as 'seen'
        ('seen' S3 are subtracted from total S3 by buildDeck to make room for new S0 terms).
    S0-2 with upgrade go to the back of their queues.
    S0-2 with downgrade go to the front of their queues.
    '''
    # ***
    change_ct = 0
    
    tqueues_mirror = copy.deepcopy(t_queues)
    
    recal_stats = collections.Counter()

    for score, term_queue in tqueues_mirror.items():

        # compare with term_dict, change queue if necessary
        for term in term_queue:

            # most current score, for comparison with old score
            cur_score = terms_dict[term]['score']

            # remove term from old queue and add to new
            if score != cur_score:

                # count for stats
                change = f'{score}->{cur_score}'
                recal_stats.update([change])

                # send to back of new queue if upgraded
                if int(cur_score) > int(score):
                    t_queues[score].remove(term)
                    t_queues[cur_score].append(term)

                    # add seen tag for S3 terms
                    if cur_score == '3':
                        terms_dict[term]['seen'] = True

                # send to front of new queue if downgraded
                elif int(cur_score) < int(score):
                    t_queues[score].remove(term)
                    t_queues[cur_score].insert(0, term)

            # move on
            else:
                continue
    
    # return new queues
    # also return terms_dict with S3 'seen' tags
    return {'terms_dict':terms_dict, 'term_queues':t_queues, 'recal_stats':recal_stats, 'mirror':tqueues_mirror}

In [4]:
test_ct = 0

for score, term_queue in term_queues.items():

        print(f'score{score} with terms: ', len(term_queue))
    
        # compare with term_dict, change queue if necessary
        for term in term_queue:
    
            # most current score, for comparison with old score
            cur_score = terms_dict[term]['score']

            # remove term from old queue and add to new
            if score != cur_score:
                
                test_ct += 1
                
test_ct

score3 with terms:  682
score1 with terms:  340
score2 with terms:  340
score0 with terms:  337


40

In [5]:
test_recal = recalibrateTermQueues(terms_dict, term_queues)

print('recal stats: ', sum(count for change, count in test_recal['recal_stats'].items()))

score3 with terms:  682
score1 with terms:  340
score2 with terms:  340
score0 with terms:  337
40
recal stats:  40


In [6]:
test_recal['recal_stats']

Counter({'0->1': 1,
         '0->2': 2,
         '0->3': 2,
         '1->0': 6,
         '1->2': 5,
         '1->3': 7,
         '2->0': 4,
         '2->1': 5,
         '2->3': 1,
         '3->1': 4,
         '3->2': 3})

In [7]:
for score, term_queue in test_recal['mirror'].items():

        print(f'score{score} with terms: ', len(term_queue))

score3 with terms:  682
score1 with terms:  340
score2 with terms:  340
score0 with terms:  337


In [8]:
for score, term_queue in test_recal['term_queues'].items():

        print(f'score{score} with terms: ', len(term_queue))

score3 with terms:  685
score1 with terms:  332
score2 with terms:  340
score0 with terms:  342


In [17]:
list(ID for ID in test_recal['mirror']['1'] if ID in deck and ID in test_recal['term_queues']['3'])

['245', '139', '228', '307', '263', '315', '328']

In [18]:
test_recal['term_queues']['3']

['917',
 '905',
 '1025',
 '1225',
 '1334',
 '1155',
 '1181',
 '1286',
 '1015',
 '1124',
 '805',
 '1265',
 '1227',
 '1263',
 '1250',
 '692',
 '1291',
 '730',
 '1048',
 '715',
 '951',
 '1275',
 '1039',
 '1146',
 '1069',
 '1085',
 '1206',
 '1325',
 '748',
 '1360',
 '763',
 '1013',
 '847',
 '849',
 '1036',
 '1318',
 '884',
 '900',
 '721',
 '722',
 '902',
 '1362',
 '1283',
 '701',
 '1043',
 '1030',
 '938',
 '1028',
 '957',
 '782',
 '927',
 '1211',
 '872',
 '1296',
 '1214',
 '1237',
 '1078',
 '724',
 '1184',
 '1348',
 '882',
 '966',
 '1164',
 '1144',
 '1243',
 '1008',
 '955',
 '1299',
 '1246',
 '1163',
 '1075',
 '1234',
 '1057',
 '843',
 '986',
 '1305',
 '1126',
 '776',
 '936',
 '1315',
 '1266',
 '1271',
 '1197',
 '1166',
 '816',
 '873',
 '907',
 '1090',
 '950',
 '790',
 '1121',
 '1259',
 '960',
 '863',
 '806',
 '734',
 '983',
 '1073',
 '923',
 '706',
 '1136',
 '824',
 '1361',
 '943',
 '1335',
 '1298',
 '1244',
 '851',
 '835',
 '788',
 '1162',
 '780',
 '1252',
 '1219',
 '811',
 '704',
 '1277